In [1]:
import matplotlib.pyplot as plt 

import numpy as np # linear algebra
import pandas as pd #data processing

from sklearn.model_selection import train_test_split

import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression# Logistic Regression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix


In [2]:
df = pd.read_csv('datasets/fake_real_final_dataset.csv')
df

,Unnamed: 0,title,label
0,0,"Dancing With The Stars 2018 Cast News, Tour an...",REAL
1,1,Amal Clooney displays her post-baby body in ti...,FAKE
2,2,Black Eyed Peas' Taboo claims Fergie turned do...,REAL
3,3,Pregnant Beyonce & Jay Z Jet Off To Bora Bora ...,REAL
4,4,Clint Eastwood Leaves His Estate To The Trump ...,FAKE
...,...,...,...
23191,23191,John Cena Plays His Entire Family (And Slimes ...,REAL
23192,23192,Chloe Moretz steps out in sharp outfit and 'f*...,REAL
23193,23193,‘NASA Confirms’ 15 Days Of Darkness Coming Thi...,FAKE
23194,23194,Paris Hilton's Happy It Girls Like Kendall Jen...,REAL


In [3]:
train, test = train_test_split(df, test_size=0.3)
train

,Unnamed: 0,title,label
17630,17630,Lea Michele Shares Emma Roberts and Darren Cri...,REAL
10229,10229,Rihanna explains how she dresses for her 'fluc...,REAL
18234,18234,Selena Gomez Believes Justin Bieber Is Her ‘So...,FAKE
1484,1484,Meet The New Plastics Aka The Cast Of Very Cav...,REAL
18933,18933,Why Are TLC Going on Tour With the Dude From S...,REAL
...,...,...,...
13542,13542,Mandy Moore Plays ‘Why is Mandy Crying’ with E...,REAL
10878,10878,The One with the Rumor,FAKE
2377,2377,Prince Harry Goes On Solo Charity Trip To Afri...,REAL
7994,7994,5 Amazing Chris Farley Moments We'll Never For...,REAL


In [4]:
print(train.shape, test.shape)

(16237, 3) (6959, 3)


In [5]:
print(train.isnull().sum())
print('************')
print(test.isnull().sum())

Unnamed: 0    0
title         0
label         0
dtype: int64
************
Unnamed: 0    0
title         0
label         0
dtype: int64


In [6]:
stop_words = stopwords.words('english')
lemmatizer=WordNetLemmatizer()
for index,row in train.iterrows():
    filter_sentence = ''
    
    sentence = row['title']
    sentence = re.sub(r'[^\w\s]','',sentence) #cleaning
    
    words = nltk.word_tokenize(sentence) #tokenization
    
    words = [w for w in words if not w in stop_words]  #stopwords removal
    
    for word in words:
        filter_sentence = filter_sentence + ' ' + str(lemmatizer.lemmatize(word)).lower()
        
    train.loc[index,'title'] = filter_sentence


/Users/zensajnani/Desktop/Zen Projects/Hackathon/fake_news_detection/env/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [7]:
train = train[['title','label']]

In [8]:
X_train = train['title']
Y_train = train['label']

In [9]:
#Feature extraction using count vectorization and tfidf.
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(X_train)
freq_term_matrix = count_vectorizer.transform(X_train)
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(freq_term_matrix)
tf_idf_matrix = tfidf.transform(freq_term_matrix)

In [12]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in iter.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [13]:
test_counts = count_vectorizer.transform(test['title'].values)
test_tfidf = tfidf.transform(test_counts)

#split in samples
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tf_idf_matrix, Y_train, random_state=0)
print(X_train.shape,X_test.shape)

(12177, 16706) (4060, 16706)


In [14]:
logreg = LogisticRegression(C=1e5)
logreg.fit(X_train, y_train)
pred = logreg.predict(X_test)
print('Accuracy of Logistic classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic classifier on test set: {:.2f}'
     .format(logreg.score(X_test, y_test)))

cm = confusion_matrix(y_test, pred)
cm

Accuracy of Logistic classifier on training set: 1.00
Accuracy of Logistic classifier on test set: 0.82


/Users/zensajnani/Desktop/Zen Projects/Hackathon/fake_news_detection/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array([[ 557,  436],
       [ 310, 2757]])

In [15]:
NB = MultinomialNB()
NB.fit(X_train, y_train)
pred = NB.predict(X_test)
print('Accuracy of NB  classifier on training set: {:.2f}'
     .format(NB.score(X_train, y_train)))
print('Accuracy of NB classifier on test set: {:.2f}'
     .format(NB.score(X_test, y_test)))
cm = confusion_matrix(y_test, pred)
cm

Accuracy of NB  classifier on training set: 0.84
Accuracy of NB classifier on test set: 0.81


array([[ 246,  747],
       [  37, 3030]])

In [16]:
X_train = train['title']
Y_train = train['label']

In [17]:
from sklearn.pipeline import Pipeline
import joblib

from sklearn import linear_model

In [18]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer(norm='l2')),
    ('clf', linear_model.LogisticRegression(C=1e5)),
])


In [19]:
pipeline.fit(X_train, Y_train)

/Users/zensajnani/Desktop/Zen Projects/Hackathon/fake_news_detection/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=100000.0))])

In [26]:
pipeline.predict(["In an interview with @greta, Sen. Rick Scott said “Joe Biden supported the Castro regime and meeting with Maduro.”"])

array(['REAL'], dtype=object)

In [22]:
#saving the pipeline
filename = 'pipeline_final.sav'
joblib.dump(pipeline, filename)

['pipeline_final.sav']

In [23]:
loaded_model = joblib.load(filename)


In [24]:
result = loaded_model.predict(["‘Tormented’ Brad Pitt will only be allowed ‘to spend four hours with his children on Christmas Day’"])
print(result)

['REAL']
